Follow along this tutorial: https://github.com/alexeygrigorev/rag-agents-workshop

In [1]:
!which pip

/home/codespace/.python/current/bin/pip


In [2]:
#!pip install minsearch

# Installa pacchetti necessari usando uv inline
%pip install --use-uv --upgrade --no-cache jupyter ipykernel markdown
%pip install --use-uv --upgrade --no-cache minsearch requests openai

# Registra un kernel associato a questo ambiente (non sarà nascosto come quello in .venv)
#!python -m ipykernel install --user --name agents-uv-inline --display-name "agents (uv inline)"




/workspaces/my-llm-zoomcamp/0a-2025-agents/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/workspaces/my-llm-zoomcamp/0a-2025-agents/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
print(sys.executable)


/workspaces/my-llm-zoomcamp/0a-2025-agents/.venv/bin/python


In [4]:
import sys
print(sys.executable)
!which python


/workspaces/my-llm-zoomcamp/0a-2025-agents/.venv/bin/python
/workspaces/my-llm-zoomcamp/0a-2025-agents/.venv/bin/python


In [5]:
import requests
print(requests.__version__)


2.32.4


In [6]:
import sys
sys.path.append('/workspaces/my-llm-zoomcamp/0a-2025-agents/')


In [7]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
        
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
from minsearch import AppendableIndex # permette di aggiungere documenti in modo incrementale

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [9]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [10]:
question = 'Can I still join the course?'

In [11]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(query, search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [12]:
search_results = search(question)

In [13]:
search_results[0]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp',
 '_id': 2}

In [14]:
prompt = build_prompt(question, search_results)

In [15]:
from openai import OpenAI
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content



In [16]:
answer = llm(prompt)

In [17]:
print(answer)

Yes, you can still join the course even if you don't register. You are eligible to submit homeworks, but be aware that there will be deadlines for turning in the final projects, so it's best not to leave everything for the last minute.


In [18]:
# RAG pipeline in una sola funzione

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [19]:
# RAG base che usa solo knowledgebase deve rispondere così
rag("How do I patch KDE under FreeBSD?")

"I'm sorry, but there is no information available in the context regarding how to patch KDE under FreeBSD."

## "Agentic" RAG

In [20]:
# qui il rag usa knowledgebase e its own knowledge

prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
{question}
</QUESTION>

<CONTEXT> 
{context}
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}
""".strip()

In [21]:
question = 'Can I still join the course?'
context = 'EMPTY'

In [22]:
prompt = prompt_template.format(question=question, context=context)

In [23]:
answer_json = llm(prompt)

In [24]:
import json

In [25]:
answer = json.loads(answer_json)

In [26]:
answer['action']

'SEARCH'

In [27]:
answer['reasoning']

'The question is about joining the course, which may depend on enrollment deadlines and policies that are typically found in the FAQ database.'

In [28]:
# somiglia a build_prompt(query, search_results) ma ha solo un input

def build_context(search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    return context.strip()

In [29]:
search_results = search(question)
context = build_context(search_results)
prompt = prompt_template.format(question=question, context=context)

In [30]:
answer_json = llm(prompt)

In [31]:
print(answer_json)

{
"action": "ANSWER",
"answer": "Yes, you can still join the course even after the start date. You are eligible to submit homework assignments, but please keep in mind that there will be deadlines for the final projects. It's important not to procrastinate, so plan accordingly to ensure you meet all requirements.",
"source": "CONTEXT"
}


In [32]:
question = 'How do I patch KDE under FreeBSD?'
context = 'EMPTY'

In [33]:
prompt = prompt_template.format(question=question, context=context)
answer_json = llm(prompt)
answer = json.loads(answer_json)


In [34]:
answer['source']

'OWN_KNOWLEDGE'

In [35]:
question = 'May you point me out the nearest gas station?'
context = 'EMPTY'

In [36]:
prompt = prompt_template.format(question=question, context=context)
answer_json = llm(prompt)
answer = json.loads(answer_json)

In [37]:
answer['action']

'SEARCH'

In [38]:
answer['reasoning']

'The question is about locating a nearest gas station, which is not covered in the current context. Therefore, I need to refer to an external FAQ database to provide accurate information.'

In [39]:
# non può rispondere con le proprie conoscenze e proviamo a cercare nella knowledgebase
search_results = search(question)
context = build_context(search_results)
prompt = prompt_template.format(question=question, context=context)

In [40]:
answer_json = llm(prompt)

In [41]:
# la ricerca nell'indice è stata infruttuosa, come si vede dalla risposta
print(answer_json)

{
"action": "SEARCH",
"reasoning": "The question is about finding a gas station, which is not related to any course content provided in the context. Additionally, the context does not contain any information that could help answer a question about local businesses or geographical inquiries."
}


In [42]:
# questa è la pipeline equivalente di rag()

def agentic_rag_v1(question):
    context = "EMPTY"
    prompt = prompt_template.format(question=question, context=context)
    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(answer)

    if answer['action'] == 'SEARCH':
        print('need to perform search...')
        search_results = search(question)
        context = build_context(search_results)
        
        prompt = prompt_template.format(question=question, context=context)
        answer_json = llm(prompt)
        answer = json.loads(answer_json)
        print(answer)

    return answer

In [43]:
# Test

# agentic_rag_v1('how do I join the course?') # Knowledgebase
agentic_rag_v1('how patch KDE under FreeBSD?') # OWN KNOWLEDGE

{'action': 'ANSWER', 'answer': "To patch KDE under FreeBSD, you'll typically follow these steps: \n1. Ensure you have the necessary development tools installed, including 'git', 'patch', and 'make'. You can install these using the pkg package manager if they're not already present.\n2. Obtain the KDE source code. This can usually be done through the FreeBSD ports system or by cloning the repository from KDE's official GitHub or hosting site.\n3. Locate the patch file you want to apply. This could be a patch provided by KDE developers, or one that you've created yourself.\n4. Navigate to the directory of the KDE source code in your terminal.\n5. Apply the patch using the 'patch' command. For example, you would run: `patch -p1 < /path/to/your/patchfile.patch`\n6. After applying the patch, you'll want to build KDE from source again, which can be done with 'make' and 'make install'.\n7. Finally, restart your KDE session to apply the changes. \n\nBe sure to check any documentation specific 

{'action': 'ANSWER',
 'answer': "To patch KDE under FreeBSD, you'll typically follow these steps: \n1. Ensure you have the necessary development tools installed, including 'git', 'patch', and 'make'. You can install these using the pkg package manager if they're not already present.\n2. Obtain the KDE source code. This can usually be done through the FreeBSD ports system or by cloning the repository from KDE's official GitHub or hosting site.\n3. Locate the patch file you want to apply. This could be a patch provided by KDE developers, or one that you've created yourself.\n4. Navigate to the directory of the KDE source code in your terminal.\n5. Apply the patch using the 'patch' command. For example, you would run: `patch -p1 < /path/to/your/patchfile.patch`\n6. After applying the patch, you'll want to build KDE from source again, which can be done with 'make' and 'make install'.\n7. Finally, restart your KDE session to apply the changes. \n\nBe sure to check any documentation specific

## Agentic Search

In [44]:
def dedup(seq):
    seen = set()
    result = []
    for el in seq:
        _id = el['_id']
        if _id in seen:
            continue
        seen.add(_id)
        result.append(el)
    return result

In [45]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than {max_iterations} iterations for a given student question.
The current iteration number: {iteration_number}. If we exceed the allowed number 
of iterations, give the best possible answer with the provided information.

Output templates:

If you want to perform search, use this template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>",
"keywords": ["search query 1", "search query 2", ...]
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER_CONTEXT",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}

<QUESTION>
{question}
</QUESTION>

<SEARCH_QUERIES>
{search_queries}
</SEARCH_QUERIES>

<CONTEXT> 
{context}
</CONTEXT>

<PREVIOUS_ACTIONS>
{previous_actions}
</PREVIOUS_ACTIONS>
""".strip()

In [46]:
question = 'how do I do well on module 1'
max_iterations = 3
iteration_number = 1
search_queries = []
search_results  = []
previous_actions = []

In [47]:
context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [48]:
answer_json = llm(prompt)

In [49]:
answer = json.loads(answer_json)
print(json.dumps(answer, indent=2))

{
  "action": "SEARCH",
  "reasoning": "To provide specific strategies and tips for doing well on Module 1, I need to gather relevant information from the FAQ database about the module's content, key areas of focus, and study strategies.",
  "keywords": [
    "how to succeed in Module 1",
    "tips for Module 1",
    "Module 1 study strategies"
  ]
}


In [50]:
previous_actions.append(answer)

In [51]:
keywords = answer['keywords']

In [52]:
for kw in keywords:
    search_queries.append(kw)
    sr = search(kw)
    search_results.extend(sr)

In [53]:
# 15 = 5 risultati per ogni ricerca, 3 ricerche (una per ogni keyword)
len(search_results)

15

In [54]:
search_results = dedup(search_results)

In [55]:
len(search_results)

6

In [56]:
iteration_number = 2

context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [57]:
answer_json = llm(prompt)

In [58]:
answer = json.loads(answer_json)
print(json.dumps(answer, indent=2))

{
  "action": "SEARCH",
  "reasoning": "To find specific advice and strategies that relate directly to succeeding in Module 1, I'll search for general best practices for learning Docker and Terraform, which are the main focuses of this module. This could include topics like key concepts to understand, common pitfalls, and effective study methods.",
  "keywords": [
    "best practices for Docker",
    "best practices for Terraform",
    "learning strategies for Docker and Terraform"
  ]
}


In [59]:
question = "what do I need to do to be successful at module 1?"

search_queries = []
search_results = []
previous_actions = []

iteration = 1

while True:
    print(f'ITERATION #{iteration}...')

    context = build_context(search_results)
    prompt = prompt_template.format(
        question=question,
        context=context,
        search_queries="\n".join(search_queries),
        previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
        max_iterations=3,
        iteration_number=iteration
    )

    print(prompt)

    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(json.dumps(answer, indent=2))

    previous_actions.append(answer)

    action = answer['action']
    if action != 'SEARCH':
        break

    keywords = answer['keywords']
    search_queries = list(set(search_queries) | set(keywords))
    
    for k in keywords:
        res = search(k)
        search_results.extend(res)

    search_results = dedup(search_results)
    
    iteration = iteration + 1
    if iteration >= 4:
        break

    print()

ITERATION #1...
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current 

In [60]:
answer

{'action': 'ANSWER',
 'answer': 'To be successful in Module 1, focus on understanding the foundational concepts of Docker and Terraform. Engage with the practical exercises provided in the course materials, and don’t hesitate to seek help when you encounter difficulties. Regularly review your work to reinforce learning, and try to collaborate with peers for a better understanding of the topics. Additionally, practicing real-world applications of Docker and Terraform will enhance your skills and confidence. Make sure you also keep up with the course timelines to maintain a steady learning pace.',
 'source': 'OWN_KNOWLEDGE'}

In [61]:
iteration

3

## Function calling ("tool use")

In [62]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [63]:
search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [64]:
# su richiesta (funcion call) di LLM, chiama la funzione search
def do_call(tool_call_response):
    function_name = tool_call_response.name
    arguments = json.loads(tool_call_response.arguments)

    f = globals()[function_name] # recupera nome della funzione dal contesto globale
    result = f(**arguments)

    return {
        "type": "function_call_output",
        "call_id": tool_call_response.call_id,
        "output": json.dumps(result, indent=2),
    }

In [65]:
question = "How do I do well in module 1?"

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.
If you look up something in FAQ, convert the student question into multiple queries.
""".strip()

tools = [search_tool] # a LLM noi passiamo solo la descrizione della funzione, non la funzione stessa

chat_messages = [
    {"role": "developer", "content": developer_prompt},
    {"role": "user", "content": question}
]

# usando end-point responses, non passiamo l'intero prompt, solo developer prompt e question
response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)
response.output

[ResponseFunctionToolCall(arguments='{"query":"how to do well in module 1"}', call_id='call_bFjD0NrjxE0XQl9Ni8AqZBmh', name='search', type='function_call', id='fc_687d47674df0819293e9d20c7c6c748c074561c8fdb9cb77', status='completed'),
 ResponseFunctionToolCall(arguments='{"query":"tips for success in module 1"}', call_id='call_84d9bYn2k3YSP60XPkbUEfSP', name='search', type='function_call', id='fc_687d4767836c8192a3ad9da7d062e294074561c8fdb9cb77', status='completed'),
 ResponseFunctionToolCall(arguments='{"query":"module 1 study strategies"}', call_id='call_kegCpKZmYbeJ7VAR5iEQT1Uu', name='search', type='function_call', id='fc_687d4767bba08192ab225a696966613f074561c8fdb9cb77', status='completed')]

In [66]:
calls = response.output

In [67]:
for call in calls:
    result = do_call(call)
    chat_messages.append(call)
    chat_messages.append(result)

In [68]:
response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)
response.output

[ResponseOutputMessage(id='msg_687d47688aa08192b223e93423a8dee7074561c8fdb9cb77', content=[ResponseOutputText(annotations=[], text='To do well in Module 1, here are some strategies and tips you can follow:\n\n1. **Understand the Basics**: Ensure you have a strong understanding of Docker and Terraform as they form significant parts of Module 1. Familiarize yourself with the foundational concepts.\n\n2. **Practice Regularly**: Engage with the course materials and hands-on projects frequently. Implementing what you\'ve learned in practice will enhance your understanding.\n\n3. **Utilize Resources**: Make use of provided resources such as documentation, tutorials, and community forums. They can be invaluable when you encounter issues.\n\n4. **Install Required Packages**: Make sure to install all necessary Python packages as indicated in the course materials. For example, if you encounter errors such as `ModuleNotFoundError: No module named \'psycopg2\'`, ensure you have the package install

In [69]:
r = response.output[0]
print(r.content[0].text)

To do well in Module 1, here are some strategies and tips you can follow:

1. **Understand the Basics**: Ensure you have a strong understanding of Docker and Terraform as they form significant parts of Module 1. Familiarize yourself with the foundational concepts.

2. **Practice Regularly**: Engage with the course materials and hands-on projects frequently. Implementing what you've learned in practice will enhance your understanding.

3. **Utilize Resources**: Make use of provided resources such as documentation, tutorials, and community forums. They can be invaluable when you encounter issues.

4. **Install Required Packages**: Make sure to install all necessary Python packages as indicated in the course materials. For example, if you encounter errors such as `ModuleNotFoundError: No module named 'psycopg2'`, ensure you have the package installed:
   - You can install it using:
     ```
     pip install psycopg2-binary
     ```
   - If you still experience issues, consider updating yo

In [70]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.
When using FAQ, perform deep topic exploration: make one request to FAQ,
and then based on the results, make more requests.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_messages = [
    {"role": "developer", "content": developer_prompt},
]

In [72]:
# non posso usare questo loop in vscode. Va in loop infinito

while True: # main Q&A loop
    question = input() # How do I do my best for module 1?
    if question == 'stop':
        break

    message = {"role": "user", "content": question}
    chat_messages.append(message)

    while True: # request-response loop - query API till get a message
        response = client.responses.create(
            model='gpt-4o-mini',
            input=chat_messages,
            tools=tools
        )

        has_messages = False
        
        for entry in response.output:
            chat_messages.append(entry)
        
            if entry.type == 'function_call':      
                print('function_call:', entry)
                print()
                result = do_call(entry)
                chat_messages.append(result)
            elif entry.type == 'message':
                print(entry.content[0].text)
                print()
                has_messages = True

        if has_messages:
            break

 May I still register for the course?


function_call: ResponseFunctionToolCall(arguments='{"query":"registration deadlines"}', call_id='call_yfw3PG92P7g6zJpTJfDstGcv', name='search', type='function_call', id='fc_687d48e9f208819da48d68e33aaba11b021fc37eaf046226', status='completed')

Yes, you can still register for the course! Even if you don't officially register, you're still eligible to participate and submit homework. Registration mainly serves to gauge interest before the course starts.

You can check the latest registration deadlines [here](https://docs.google.com/spreadsheets/d/e/2PACX-1vQACMLuutV5rvXg5qICuJGL-yZqIV0FBD84CxPdC5eZHf8TfzB-CJT_3Mo7U7oGVTXmSihPgQxuuoku/pubhtml) for any updates or extensions.

If you have any other questions about the course or participation, feel free to ask! Would you like information on specific topics covered in the course?



 stop


## Multiple tools

In [ ]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

In [73]:
def add_entry(question, answer):
    doc = {
        'question': question,
        'text': answer,
        'section': 'user added',
        'course': 'data-engineering-zoomcamp'
    }
    index.append(doc)

In [74]:
add_entry_description = {
    "type": "function",
    "name": "add_entry",
    "description": "Add an entry to the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question to be added to the FAQ database",
            },
            "answer": {
                "type": "string",
                "description": "The answer to the question",
            }
        },
        "required": ["question", "answer"],
        "additionalProperties": False
    }
}

In [75]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(search, search_tool)

In [76]:
tools.add_tool(add_entry, add_entry_description)

In [77]:
tools.get_tools()

[{'type': 'function',
  'name': 'search',
  'description': 'Search the FAQ database',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'Search query text to look up in the course FAQ.'}},
   'required': ['query'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'add_entry',
  'description': 'Add an entry to the FAQ database',
  'parameters': {'type': 'object',
   'properties': {'question': {'type': 'string',
     'description': 'The question to be added to the FAQ database'},
    'answer': {'type': 'string', 'description': 'The answer to the question'}},
   'required': ['question', 'answer'],
   'additionalProperties': False}}]

In [78]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [ ]:
chat.run()

You: How may I mange virtual environments?


In [ ]:
index.docs[-1]

In [ ]:
index